In [ ]:
WEATHER_API_KEY='52c94fc20a582a91a0994782b8ca2acc'

In [ ]:
'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}'

'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}'

In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
upload=files.upload()

Saving Updated_Datasettt.csv to Updated_Datasettt.csv


In [ ]:
import pandas as pd

# Load dataset from a local CSV file
df = pd.read_csv('Updated_Datasettt.csv')

# Check the data
df.head()


,Industry Name,Latitude,Longitude,Industry Type,Risk Category/Level,Hazardous Material Storage,Gases
0,Global Corp,-86.295922,-32.438666,Pipeline Terminal,Medium,No,"Methane, Ethane, Propane"
1,Energy Resources,-42.631386,-44.990412,Gas Plant,High,Yes,"Methane, Hydrogen Sulfide, Carbon Dioxide"
2,Industrial Corp,70.979944,178.757830,Gas Plant,Medium,Yes,"Methane, Hydrogen Sulfide, Carbon Dioxide"
3,Industrial Oil,87.732153,79.392150,Oil Rig,Medium,Yes,"Hydrogen Sulfide, Methane, Carbon Monoxide"
4,Trans Industries,-41.124096,-161.159867,Pipeline Terminal,High,Yes,"Methane, Ethane, Propane"


In [ ]:
# Select row by index
n=int(input("Enter the Number"))
selected_row = df.iloc[n]
#Name=input("Search by Industry name")
#selected_row = df[df['Industry Name'] == Name]

lat = selected_row['Latitude']
lon = selected_row['Longitude']

# Now proceed with your weather API call and other calculations
url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}'


Enter the Number4


'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}'

In [ ]:
response = requests.get(url)

In [ ]:
wind_speed = response.json()['wind']['speed']
weather = response.json()['weather'][0]
sunset_time = response.json()['sys']['sunset']
sunrise_time = response.json()['sys']['sunrise']
cloud_cover = response.json()['clouds']['all']

Insolation: \begin{align}
        Q = S (\frac{r}{r_0})^2 \cos θ_0
    \end{align}

Q is the daytime insolation (in watts per square meter)

S is the solar constant (1361 watts per square meter)

r is the Earth's distance from the Sun (in meters)

ro is the Earth's mean distance from the Sun (149.6 million meters)

 is the solar zenith angle (the angle between the sun's rays and the vertical)

\begin{align}
r = r_0 (1 + 0.0167 \cos(360 * \frac{d-3}{365}))
\end{align}
d is the number of days since January 1st
\begin{align}
θ_0 = \arccos( \sin (latitude) * \sin (\delta) + \cos(latitude) * \cos (\delta)  * \cos (hour angle) )
\end{align}


\begin{align}
hourangle = (time of day in hours - 12) * 15°
\end{align}
\begin{align}
\delta = 23.45° * \sin(360° * \frac{N - 81}{365})
\end{align}
N is the day of the year (1-365)



Strong > 100

Moderate 50-100

Slight < 50

In [ ]:
from datetime import date, datetime

today = date.today()

In [ ]:
N = pd.Period(today, freq='D').dayofyear
delta = 23.45 * np.sin(np.deg2rad(360 * (N-81) / 365))
timeofdayinhrs = datetime.now().hour
hourangle = (timeofdayinhrs - 12) * 15
theta_not = np.arccos(np.sin(lat) * np.sin(delta) + np.cos(lat) * np.cos(delta) * np.cos(hourangle))
d = (date.today() - date(date.today().year,1,1)).days
r_by_r0 = (1 + 0.0167*np.cos(360*(d-3)/365))

In [ ]:
Q = 1361 * r_by_r0**2 * np.cos(theta_not)
#Q is Insolation


In [ ]:
daytime_insolation_class = ''

if Q > 100:
    daytime_insolation_class = 'strong'
elif Q > 50 and Q <= 100:
    daytime_insolation_class = 'moderate'
else:
    daytime_insolation_class = 'slight'

In [ ]:
Q, daytime_insolation_class


(1044.0000634387964, 'strong')

Nighttime Conditions

In [ ]:
nighttime_cond = ''
if cloud_cover >= 80:
    nighttime_cond = 'overcast'
else:
    nighttime_cond = 'not overcast'

In [ ]:
cloud_cover, nighttime_cond


(27, 'not overcast')

In [ ]:
datetime.utcfromtimestamp(sunrise_time).hour


15

In [ ]:
if datetime.now().hour >= datetime.utcfromtimestamp(sunrise_time).hour + 1 and datetime.now().hour <= datetime.utcfromtimestamp(sunset_time).hour - 1:
    if wind_speed < 2:
        if daytime_insolation_class == 'strong':
            stability_class = 'A'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'B'
        else:
            stability_class = 'B'
    if wind_speed >= 2 and wind_speed < 3:
        if daytime_insolation_class == 'strong':
            stability_class = 'B'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'B'
        else:
            stability_class = 'C'
    if wind_speed >= 3 and wind_speed < 4:
        if daytime_insolation_class == 'strong':
            stability_class = 'B'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'C'
        else:
            stability_class = 'C'
    if wind_speed >= 4 and wind_speed < 6:
        if daytime_insolation_class == 'strong':
            stability_class = 'C'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'D'
        else:
            stability_class = 'D'
    if wind_speed >= 6:
        if daytime_insolation_class == 'strong':
            stability_class = 'C'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'D'
        else:
            stability_class = 'D'
else:
    if wind_speed < 2:
        stability_class = 'F'
    if wind_speed >= 2 and wind_speed < 3:
        if nighttime_cond == 'overcast':
            stability_class = 'F'
        else:
            stability_class = 'E'
    if wind_speed >= 3 and wind_speed < 4:
        if nighttime_cond == 'overcast':
            stability_class = 'E'
        else:
            stability_class = 'D'
    if wind_speed >= 4 and wind_speed < 6:
        stability_class = 'D'
    if wind_speed >= 6:
        stability_class = 'D'

In [ ]:
stability_class


'D'


determining r

In [ ]:
E_tnt = 4686 # kJ/kg
m = 1 # kg => needed to be determined from plant data (may be database or sensor)
eta = 0.15 # explosion efficiency => needed to be estimated but took the worst case where the range would be max.
delHc = 0
pa = 0  # kPa
po = 0     # peak side-on overpressure value from sensor or database (example)
if pa != 0:
    ps = po / pa
else:
    ps = float('inf')  # or handle it appropriately, maybe assign a default value


Puff model

In [ ]:
m_tnt = (eta*m*delHc)/E_tnt
ze = (ps/16.2499)**(-1/1.8093) # m/kg^(1/3)
r = ze*(m_tnt**(1/3))

**Gas Classification**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

df_new = pd.read_csv('Updated_Datasettt.csv')

N =   int(input("Enter the row"))

selected_row = df_new.iloc[[N]]

if 'Gases' in selected_row.columns and isinstance(selected_row['Gases'].iloc[0], str):
    selected_row['Gases'] = selected_row['Gases'].fillna('Unknown')
    df_gases = selected_row['Gases'].str.get_dummies(sep=', ')
else:
    df_gases = pd.DataFrame()

if df_gases.empty:
    df_gases = pd.DataFrame(columns=['Gases_Unknown'])

selected_row['Explosion_Risk'] = selected_row['Risk Category/Level'].apply(lambda x: 1 if x == 'High' else 0)

df_processed = pd.concat([df_gases, selected_row['Explosion_Risk']], axis=1)

X = df_processed.drop('Explosion_Risk', axis=1)
y = df_processed['Explosion_Risk']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_scaled, y)

# Prediction and reporting
y_pred = clf.predict(X_scaled)
report = classification_report(y, y_pred, zero_division=0)
accuracy = accuracy_score(y, y_pred)

print("Classification Report:\n", report)
print("Accuracy:", accuracy)


Enter the row4
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

Accuracy: 1.0


<ipython-input-22-5a3e8aea6b43>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_row['Gases'] = selected_row['Gases'].fillna('Unknown')
<ipython-input-22-5a3e8aea6b43>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_row['Explosion_Risk'] = selected_row['Risk Category/Level'].apply(lambda x: 1 if x == 'High' else 0)
